In [20]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
import numpy as np

# Setting up the Project

## Loading the Dataset

In [17]:
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

## Preparing the Dataset

In [21]:
# Normalizing the dataset 
x_train = x_train.reshape(x_train.shape[0], 28,28, 1).astype('float32')
x_train = ( x_train - 127.5) / 127.5 

# Adding channels dimension
x_train = np.expand_dims(x_train, axis = 3)


## Displaying the Dataset

In [ ]:
fig = pyplot.gcf()
fig.set_size_inches(25, 10.5)
fig.savefig('test2png.png', dpi=100)

for i in range(9):
	pyplot.subplot(1,9,i + 1)
	pyplot.imshow(x_train[i],cmap=pyplot.get_cmap('gray'))
# show the figure
pyplot.show()

## Create Batches

In [19]:
train_images = tf.data.Dataset.from_tensor_slices(x_train).shuffle(len(x_train)).batch(256)

# Generating the GAN

In [6]:
from tensorflow.keras import layers

## Creating the Discriminator

The purpose of a discriminator in a GAN is to determine how realistic an image looks. To do this both the real images from the training set, and generated images from the generator are passed into this model. By doing this the model is able to learn how to predict the likelihood that an image is real or not. This feature is very important, because it allows us to make sure that our generator produces realistic images. 

This discriminator takes in a 28 x 28 x 1 image and at the end will ouput the probabillity that an image is real. 

In [23]:
def discriminator():
    # Defining the model 
    model = tf.keras.Sequential()

    # Input Layer
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 1]))
    # Hidden Layers
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Output Layer
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))

    # Define Optimization function
    opt = tf.keras.optimizers.Adam(lr=2e-4, beta_1=0.5)

    # Compile Model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model